# Price–News Join Analysis

Analyze the GDELT–OHLCV join table: **news on day t → prices on day t+1** (next trading day).  
We evaluate and validate the data, then compute mean and median sentiment (and optional price metrics) per ticker per day.



In [ ]:
import pandas as pd
from pathlib import Path

# Find project root
current = Path.cwd()
while not (current / "data").exists() and current != current.parent:
    current = current.parent
PROJECT_ROOT = current
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
INPUT_PATH = PROCESSED_DIR / "gdelt_ohlcv_join.csv"

df = pd.read_csv(
    INPUT_PATH,
    parse_dates=["seendate", "article_date", "price_date"],
)
# Required for analysis
required = ["sentiment_score", "ticker", "article_date", "price_date"]
missing = [c for c in required if c not in df.columns]
assert not missing, f"Missing columns: {missing}"
print(f"Loaded {len(df):,} rows from {INPUT_PATH.name}")
print(f"Columns: {list(df.columns)}")

Loaded 6,530 rows from gdelt_ohlcv_join.csv
Columns: ['seendate', 'url', 'title', 'language', 'domain', 'socialimage', 'company', 'ticker', 'date', 'sentiment_score', 'sentiment_hits', 'sentiment_present', 'article_date', 'price_date', 'next_open', 'next_high', 'next_low', 'next_close', 'next_adj_close', 'next_volume']


## 1A. Alternative Join (Trading-Day Indexed)

Alternative approach for comparison:
- Use `prices_daily_accumulated.csv` as the left/base table (one row per trading day per ticker).
- Left-join `gdelt_articles_with_sentiment.csv` on `ticker` and calendar date.
- This keeps the trading-day index intact and attaches any same-day news rows.

This is separate from the existing `gdelt_ohlcv_join.csv` logic (`news t -> prices t+1`) and is intended for validation/EDA comparisons.

In [ ]:
# Trading-day indexed left join: prices (left) <- gdelt accumulated
prices_path = PROCESSED_DIR / "prices_daily_accumulated.csv"
gdelt_acc_path = PROCESSED_DIR / "gdelt_articles_with_sentiment.csv"

prices_left = pd.read_csv(prices_path, parse_dates=["date"])
gdelt_acc = pd.read_csv(gdelt_acc_path, parse_dates=["seendate"])

# Normalize join keys to calendar day + ticker
prices_left["trading_date"] = prices_left["date"].dt.tz_localize(None).dt.normalize()
gdelt_acc["article_date"] = pd.to_datetime(gdelt_acc["seendate"], utc=True).dt.tz_localize(None).dt.normalize()

prices_left["ticker"] = prices_left["ticker"].astype(str).str.upper().str.strip()
gdelt_acc["ticker"] = gdelt_acc["ticker"].astype(str).str.upper().str.strip()

alt_join = prices_left.merge(
    gdelt_acc,
    how="left",
    left_on=["trading_date", "ticker"],
    right_on=["article_date", "ticker"],
    suffixes=("_price", "_news"),
)

print("Alternative join built:")
print(f"  prices rows (left base): {len(prices_left):,}")
print(f"  joined rows: {len(alt_join):,}")
print(f"  trading date range: {prices_left['trading_date'].min().date()} -> {prices_left['trading_date'].max().date()}")

# Coverage diagnostics: how many trading-day x ticker rows have at least one matched article
coverage = (
    alt_join.groupby(["trading_date", "ticker"], as_index=False)
    .agg(has_news=("url", lambda s: s.notna().any()), article_rows=("url", "count"))
)

print(f"  unique (trading_date, ticker) pairs: {len(coverage):,}")
print(f"  pairs with >=1 matched article: {int(coverage['has_news'].sum()):,}")
print(f"  pairs with 0 matched articles: {int((~coverage['has_news']).sum()):,}")

print("\nSample rows from alternative join:")
display(
    alt_join[["trading_date", "ticker", "close", "seendate", "title", "sentiment_score"]]
    .sort_values(["trading_date", "ticker", "seendate"], na_position="last")
    .head(20)
)

# Keep as named object for downstream analysis cells if needed
alt_join_df = alt_join

Alternative join built:
  prices rows (left base): 231
  joined rows: 5,837
  trading date range: 2025-12-29 -> 2026-02-13
  unique (trading_date, ticker) pairs: 231
  pairs with >=1 matched article: 167
  pairs with 0 matched articles: 64

Sample rows from alternative join:


,trading_date,ticker,close,seendate,title,sentiment_score
0,2025-12-29,AAPL,273.760010,NaT,NaN,NaN
1,2025-12-29,AMZN,232.070007,NaT,NaN,NaN
2,2025-12-29,GOOGL,313.559998,NaT,NaN,NaN
3,2025-12-29,META,658.690002,NaT,NaN,NaN
4,2025-12-29,MSFT,487.100006,NaT,NaN,NaN
5,2025-12-29,NVDA,188.220001,NaT,NaN,NaN
6,2025-12-29,TSLA,459.640015,NaT,NaN,NaN
7,2025-12-30,AAPL,273.079987,NaT,NaN,NaN
8,2025-12-30,AMZN,232.529999,NaT,NaN,NaN
9,2025-12-30,GOOGL,313.850006,NaT,NaN,NaN


In [ ]:
merged = alt_join_df.sort_values(['ticker', 'trading_date'])

# Create next day's close based on each ticker's last close
merged['next_close'] = merged.groupby('ticker')['close'].shift(-1)
# Calculate next day's return
merged['next_return'] = (merged['next_close'] - merged['close']) / merged['close']
# Calculate absolute return
merged['abs_return'] = merged['next_return'].abs()

daily = merged.groupby(['ticker', 'trading_date']).agg(
    mean_sentiment=('sentiment_score', 'mean'),
    median_sentiment=('sentiment_score', 'median'),
    sentiment_count=('sentiment_present', 'sum'),
    article_count=('title', 'count'),
    next_return=('next_return', 'first'),
    abs_return=('abs_return', 'first')
).reset_index()
daily

,ticker,trading_date,mean_sentiment,median_sentiment,sentiment_count,article_count,next_return,abs_return
0,AAPL,2025-12-29,NaN,NaN,0,0,-0.002484,0.002484
1,AAPL,2025-12-30,NaN,NaN,0,0,-0.004468,0.004468
2,AAPL,2025-12-31,NaN,NaN,0,0,-0.003127,0.003127
3,AAPL,2026-01-02,NaN,NaN,0,0,-0.013837,0.013837
4,AAPL,2026-01-05,0.417500,0.38,4,8,0.000000,0.000000
...,...,...,...,...,...,...,...,...
226,TSLA,2026-02-09,0.592727,0.96,11,11,0.000000,0.000000
227,TSLA,2026-02-10,0.151333,0.00,6,15,0.000000,0.000000
228,TSLA,2026-02-11,0.007895,0.00,8,19,0.000000,0.000000
229,TSLA,2026-02-12,0.030000,0.00,5,27,0.000000,0.000000


### Questions:
- Does sentiment predict price direction?
- Does sentiment predict volatility?

### Approaches:
- Sentiment bucket analysis?
- News volume effect
- Disagreement signal (sentiment range vs abs return)

## 1. Data evaluation and validation

**First objective: validate the matching between prices and news** (join integrity, then schema and coverage).

In [ ]:
# Schema and shape
print("Shape:", df.shape)
print("\nDtypes:")
print(df.dtypes)
print("\nKey columns present:")
key_cols = ["article_date", "price_date", "ticker", "sentiment_score", "next_close", "next_volume"]
for c in key_cols:
    print(f"  {c}: {c in df.columns}")

Shape: (6530, 22)

Dtypes:
seendate             datetime64[us, UTC]
url                                  str
title                                str
language                             str
domain                               str
socialimage                          str
company                              str
ticker                               str
date                                 str
sentiment_score                  float64
sentiment_hits                   float64
sentiment_present                   bool
article_date              datetime64[us]
price_date                datetime64[us]
next_open                        float64
next_high                        float64
next_low                         float64
next_close                       float64
next_adj_close                   float64
next_volume                        int64
_gap_days                          int64
days_to_next                       int64
dtype: object

Key columns present:
  article_date: True
  price_date: 

### 1.1 Validate price–news matching

Check that **price_date** is the next trading day after **article_date** and that attached prices match the source OHLCV.

In [ ]:
# 1) price_date must be strictly after article_date (next trading day)
df["_gap_days"] = (df["price_date"] - df["article_date"]).dt.days
bad_order = (df["_gap_days"] <= 0).sum()
print("1) Article date → price date (next trading day)")
# Display number of misaligned rows (s/b 0)
print(f"   Rows where price_date ≤ article_date: {bad_order} (expect 0)")
# Unit test for correct matching
assert bad_order == 0, "Every row must have price_date > article_date"
print("   ✓ All rows have price_date after article_date")
print("\n   Calendar-day gap (article_date to price_date):")
print(df["_gap_days"].value_counts().sort_index().to_string())

1) Article date → price date (next trading day)
   Rows where price_date ≤ article_date: 0 (expect 0)
   ✓ All rows have price_date after article_date

   Calendar-day gap (article_date to price_date):
_gap_days
1    5471
2     356
3     592
4     111


In [ ]:
# 2) Within join: each (price_date, ticker) should have exactly one set of next_* values (no conflicts)
price_cols = [c for c in df.columns if c.startswith("next_")]
check = df.groupby(["price_date", "ticker"])[price_cols].nunique()
max_per_col = check.max()
conflicts = (check > 1).any(axis=1).sum()
print("2) One price per (price_date, ticker)")
print(f"   Unique (price_date, ticker) pairs: {len(check):,}")
print(f"   Pairs with conflicting next_* values: {conflicts} (expect 0)")
# Unit test for unique prices per (price_date, ticker)
assert conflicts == 0, "⚠ Some (price_date, ticker) have multiple different prices — investigate"
print("   ✓ All rows for same (price_date, ticker) have identical next_* values")

2) One price per (price_date, ticker)
   Unique (price_date, ticker) pairs: 176
   Pairs with conflicting next_* values: 0 (expect 0)
   ✓ All rows for same (price_date, ticker) have identical next_* values


In [ ]:
# 3) Cross-check: join next_* values vs source OHLCV (prices_daily_accumulated)
ohlcv_path = PROCESSED_DIR / "prices_daily_accumulated.csv"
if ohlcv_path.exists():
    ohlcv = pd.read_csv(ohlcv_path, parse_dates=["date"])
    # One row per (date, ticker) in join; take first next_* per (price_date, ticker)
    join_prices = df.groupby(["price_date", "ticker"])["next_close"].first().reset_index()
    join_prices = join_prices.rename(columns={"price_date": "date", "next_close": "join_close"})
    merged = join_prices.merge(ohlcv[["date", "ticker", "close"]], on=["date", "ticker"], how="left")
    merged["match"] = merged["join_close"].round(6) == merged["close"].round(6)
    mismatches = (~merged["match"]).sum()
    missing = merged["close"].isna().sum()
    print("3) Join vs source OHLCV (next_close vs close)")
    print(f"   (price_date, ticker) pairs checked: {len(merged):,}")
    print(f"   Mismatches (join next_close ≠ OHLCV close): {mismatches}")
    print(f"   Missing in OHLCV: {missing}")
    # Unit test for matching prices
    assert mismatches == 0 and missing == 0, "   ⚠ Review mismatches or missing dates"
    print("   ✓ All join prices match source OHLCV")
else:
    print("3) Skip cross-check (prices_daily_accumulated.csv not found)")

3) Join vs source OHLCV (next_close vs close)
   (price_date, ticker) pairs checked: 176
   Mismatches (join next_close ≠ OHLCV close): 0
   Missing in OHLCV: 0
   ✓ All join prices match source OHLCV


### 1.2 Schema, date ranges, and coverage

Schema, date ranges, missing values, and ticker coverage (for downstream aggregation).

In [ ]:
# Date ranges and join alignment
art_min, art_max = df["article_date"].min(), df["article_date"].max()
price_min, price_max = df["price_date"].min(), df["price_date"].max()
print("Article date range:", art_min.date(), "to", art_max.date())
print("Price date range: ", price_min.date(), "to", price_max.date())
print("\nExpected: price_date = next trading day after article_date (weekends/holidays skipped).")
# Spot-check: article_date and price_date should differ by 1–3 calendar days (Fri→Mon = 3)
df["days_to_next"] = (df["price_date"] - df["article_date"]).dt.days
print("\nCalendar days from article_date to price_date (sample):")
print(df["days_to_next"].value_counts().head(10))

Article date range: 2026-01-05 to 2026-02-12
Price date range:  2026-01-06 to 2026-02-13

Expected: price_date = next trading day after article_date (weekends/holidays skipped).

Calendar days from article_date to price_date (sample):
days_to_next
1    5471
3     592
2     356
4     111
Name: count, dtype: int64


In [ ]:
# Missing values in columns used for aggregation
agg_cols = ["sentiment_score", "ticker", "article_date", "price_date"]
if "next_close" in df.columns:
    agg_cols.append("next_close")
missing = df[agg_cols].isna().sum()
print("Missing values (columns used for mean/median per ticker per day):")
print(missing[missing > 0] if missing.any() else "  None")
print("\nRows with any missing in these columns:", df[agg_cols].isna().any(axis=1).sum())

Missing values (columns used for mean/median per ticker per day):
  None

Rows with any missing in these columns: 0


In [ ]:
# Ticker coverage: articles and (article_date, ticker) pairs
print("Articles per ticker:")
print(df["ticker"].value_counts().sort_index())
print("\nUnique (article_date, ticker) pairs per ticker = distinct calendar days with ≥1 article:")
# Per ticker, count distinct article_date (same as count of (article_date, ticker) per ticker)
unique_days_per_ticker = df.groupby("ticker")["article_date"].nunique()
print(unique_days_per_ticker.to_string())
print("\n(Multiple articles per day per ticker is expected; we aggregate to mean/median per (date, ticker) later.)")

Articles per ticker:
ticker
AAPL      700
AMZN      745
GOOGL    1277
META     1112
MSFT      729
NVDA     1462
TSLA      505
Name: count, dtype: int64

Unique (article_date, ticker) pairs per ticker = distinct calendar days with ≥1 article:
ticker
AAPL     27
AMZN     30
GOOGL    36
META     36
MSFT     26
NVDA     28
TSLA     30

(Multiple articles per day per ticker is expected; we aggregate to mean/median per (date, ticker) later.)


In [ ]:
# Compute mean/median sentiment per ticker per day
sentiment_per_ticker_per_day = df.groupby(["ticker", "price_date"])["sentiment_score"].agg(["mean", "median"])
print("\nMean and median sentiment per ticker per day:")
print(sentiment_per_ticker_per_day.head())

# Sanity check aggregated outputs across tickers



Mean and median sentiment per ticker per day:
                       mean  median
ticker price_date                  
AAPL   2026-01-06  0.417500    0.38
       2026-01-07  0.262857    0.00
       2026-01-08  0.052500    0.00
       2026-01-12  0.317308    0.00
       2026-01-13  0.230213    0.00


## 2. Sentiment summary statistics per ticker per day

Aggregate to mean and median sentiment per (ticker, day).

In [ ]:
# Aggregate: one row per (article_date, ticker)
daily = df.groupby(["article_date", "ticker"]).agg(
    sentiment_mean=("sentiment_score", "mean"),
    sentiment_median=("sentiment_score", "median"),
    article_count=("sentiment_score", "count"),
).reset_index()
if "next_close" in df.columns:
    daily["next_close"] = df.groupby(["article_date", "ticker"])["next_close"].first().values
if "next_volume" in df.columns:
    daily["next_volume"] = df.groupby(["article_date", "ticker"])["next_volume"].first().values

print("Daily summary (first rows):")
daily.head(10)

Daily summary (first rows):


,article_date,ticker,sentiment_mean,sentiment_median,article_count,next_close,next_volume
0,2026-01-05,AAPL,0.417500,0.38,8,262.359985,52352100
1,2026-01-05,AMZN,0.380000,0.38,2,240.929993,53764700
2,2026-01-05,GOOGL,0.910000,0.91,2,314.339996,31212100
3,2026-01-05,MSFT,0.006667,0.00,6,478.510010,23037700
4,2026-01-05,NVDA,0.411429,0.00,7,187.240005,176862600
5,2026-01-05,TSLA,0.066667,0.00,3,432.959991,89093800
6,2026-01-06,AAPL,0.262857,0.00,21,260.329987,48309800
7,2026-01-06,AMZN,0.206923,0.00,52,241.559998,42236500
8,2026-01-06,GOOGL,0.490769,0.76,13,321.980011,35104400
9,2026-01-06,META,0.125000,0.00,16,648.690002,12846300


## 3. Monday-only analysis: sentiment per ticker per week

Filter to **Mondays only** (article_date), then group by week (7-day increments starting from **2026-01-12** as week 1). Compute summary statistics per ticker per week (open-ended; weeks 1+).

In [ ]:
# Filter to Mondays only (dayofweek: Monday=0)
mondays = daily[daily["article_date"].dt.dayofweek == 0].copy()
print(f"Total rows: {len(daily):,}")
print(f"Mondays only: {len(mondays):,} ({100*len(mondays)/len(daily):.1f}%)")
print(f"\nMonday dates in data:")
monday_dates = sorted(mondays["article_date"].dt.date.unique())
print(monday_dates)
print(f"\nExpected Mondays (if all weeks present):")
week_start = pd.Timestamp("2026-01-12")
for w in range(1, 6):  # weeks 1-5 based on data up to 2/9
    expected_monday = week_start + pd.Timedelta(days=7*(w-1))
    print(f"  Week {w}: {expected_monday.date()}")
    if expected_monday.date() not in monday_dates:
        print(f"    ⚠ MISSING from data")

Total rows: 213
Mondays only: 41 (19.2%)

Monday dates in data:
[datetime.date(2026, 1, 5), datetime.date(2026, 1, 12), datetime.date(2026, 1, 19), datetime.date(2026, 1, 26), datetime.date(2026, 2, 2), datetime.date(2026, 2, 9)]

Expected Mondays (if all weeks present):
  Week 1: 2026-01-12
  Week 2: 2026-01-19
  Week 3: 2026-01-26
  Week 4: 2026-02-02
  Week 5: 2026-02-09


In [ ]:
# Assign week number (increments of 7 days starting from 2026-01-12)
# Week 1 = 2026-01-12 + 0-6 days, Week 2 = 2026-01-12 + 7-13 days, etc.
week_start_date = pd.Timestamp("2026-01-12")
mondays["days_since_week1_start"] = (mondays["article_date"] - week_start_date).dt.days
mondays["week"] = (mondays["days_since_week1_start"] // 7) + 1
# Explain missing weeks (MLK holiday on 1/19)
print("Note: MLK holiday on 1/19 causes week 2 to be missing")
# Filter to weeks >= 1 (exclude dates before 1/12, but no upper limit - open-ended)
mondays_filtered = mondays[mondays["week"] >= 1].copy()
print(f"Week start date: {week_start_date.date()}")
print(f"Week range in data: {mondays_filtered['week'].min()} to {mondays_filtered['week'].max()} (open-ended)")
print(f"\nMondays per week:")
week_counts = mondays_filtered.groupby("week")["article_date"].nunique()
print(week_counts)
print(f"\nMissing weeks (expected but not present):")
all_weeks = set(range(mondays_filtered['week'].min(), mondays_filtered['week'].max() + 1))
present_weeks = set(week_counts.index)
missing_weeks = sorted(all_weeks - present_weeks)
if missing_weeks:
    for w in missing_weeks:
        expected_date = week_start_date + pd.Timedelta(days=7*(w-1))
        print(f"  Week {w}: {expected_date.date()} (no articles on this Monday)")
        # Check if this date exists in the raw daily data (not just Mondays)
        if expected_date.date() in daily["article_date"].dt.date.values:
            print(f"    → Date exists in daily data but is not a Monday (dayofweek check)")
        else:
            print(f"    → Date not in daily data at all (no articles on this date)")
else:
    print("  None")

Note: MLK holiday on 1/19 causes week 2 to be missing
Week start date: 2026-01-12
Week range in data: 1 to 5 (open-ended)

Mondays per week:
week
1    1
2    1
3    1
4    1
5    1
Name: article_date, dtype: int64

Missing weeks (expected but not present):
  None


In [ ]:
# Summary statistics per ticker per week (Mondays only)
# Use mondays_filtered to exclude week 0 (dates before 1/12)
weekly_stats = mondays_filtered.groupby(["ticker", "week"]).agg(
    sentiment_mean=("sentiment_mean", "mean"),
    sentiment_median=("sentiment_median", "median"),
    # sentiment_std excluded: std of already-aggregated daily means is problematic (NaN when only one Monday)
    monday_count=("article_date", "nunique"),  # number of Mondays in this week with data
    total_articles=("article_count", "sum"),
).reset_index()

print("Summary statistics per ticker per week (Mondays only, weeks 1+):")
print("=" * 80)
weekly_stats

Summary statistics per ticker per week (Mondays only, weeks 1+):


,ticker,week,sentiment_mean,sentiment_median,monday_count,total_articles
0,AAPL,1,0.230213,0.00,1,47
1,AAPL,2,0.008947,0.00,1,19
2,AAPL,3,0.212051,0.00,1,39
3,AAPL,4,0.000000,0.00,1,1
4,AAPL,5,0.180541,0.00,1,37
5,AMZN,1,0.184828,0.00,1,29
6,AMZN,2,0.071600,0.00,1,25
7,AMZN,3,0.120217,0.00,1,46
8,AMZN,4,-0.190000,0.00,1,4
9,AMZN,5,0.171714,0.00,1,35


In [ ]:
# Pivot table for mean sentiment per ticker (per week) - weeks 1+ (open-ended)
weekly_pivot_mean = weekly_stats.pivot(index="ticker", columns="week", values="sentiment_mean")
print("Mean sentiment per ticker per week (Mondays only, weeks 1+) - pivot view:")
print("=" * 80)
print("Note: Week 0 (dates before 1/12) is excluded. If a ticker has NaN for a week,")
print("      that ticker had no articles on the Monday(s) in that week.")
print("      Weeks are open-ended; new weeks will appear as data extends past 2/9.")
weekly_pivot_mean


Mean sentiment per ticker per week (Mondays only, weeks 1+) - pivot view:
Note: Week 0 (dates before 1/12) is excluded. If a ticker has NaN for a week,
      that ticker had no articles on the Monday(s) in that week.
      Weeks are open-ended; new weeks will appear as data extends past 2/9.


week,1,2,3,4,5
ticker,,,,,
AAPL,0.230213,0.008947,0.212051,0.000000,0.180541
AMZN,0.184828,0.071600,0.120217,-0.190000,0.171714
GOOGL,0.110455,0.137143,0.224483,-0.016667,0.148986
META,0.000968,0.350769,0.054074,0.083793,0.024571
MSFT,-0.024211,0.199565,0.193191,0.000000,0.136000
NVDA,0.370000,0.185909,0.313163,0.000000,0.206707
TSLA,0.098125,0.330909,0.224000,-0.910000,0.592727


In [ ]:
# Pivot table for median sentiment per ticker (per week)
weekly_pivot_median = weekly_stats.pivot(index="ticker", columns="week", values="sentiment_median")
print("Median sentiment per ticker per week (Mondays only) - pivot view:")
print("=" * 70)
# Display table
weekly_pivot_median

Median sentiment per ticker per week (Mondays only) - pivot view:


week,1,2,3,4,5
ticker,,,,,
AAPL,0.00,0.0,0.0,0.00,0.00
AMZN,0.00,0.0,0.0,0.00,0.00
GOOGL,0.00,0.0,0.0,0.00,0.00
META,0.00,0.0,0.0,0.00,0.00
MSFT,0.00,0.0,0.0,0.00,0.00
NVDA,0.12,0.0,0.0,0.00,0.00
TSLA,0.00,0.0,0.0,-0.91,0.96
